In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import statsmodels.api as sm
import json
import time
from getpass import getpass
from sklearn.preprocessing import RobustScaler

import wrds_loader
import hrp
import hrp_utils
import regression

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

import warnings
warnings.filterwarnings("ignore")

In [3]:
import importlib
importlib.reload(hrp)
importlib.reload(hrp_utils)

<module 'hrp_utils' from '/Users/vdp/mfe/quarters/quarter_4_fall_2023/AFP/hierarchical_risk_parity/hrp_utils.py'>

In [4]:
# directory information
base_path = os.path.dirname(os.getcwd())
data_folder = os.path.join(base_path, "data")
processed_data_folder = os.path.join(data_folder, "processed_data")
returns_folder = os.path.join(data_folder, "returns")
returns_files = os.listdir(returns_folder)

results_folder = os.path.join(data_folder, "results")

factors_folder = os.path.join(data_folder, "finratios")
mom_folder = os.path.join(data_folder, "momentum")
reversal_folder = os.path.join(data_folder, "reversal")

# data file
ret_file = os.path.join(processed_data_folder, "all_data.pkl")
cluster_file = os.path.join(processed_data_folder, "agglo_clusters.pkl")
factor_file = os.path.join(processed_data_folder, "factor_data.feather")

In [7]:
# WRDS login information
wrds_id = getpass("wrds id: ")
# wrds_pwd = getpass("wrds password: ")

wrds id:  ········


## Processing GICS Data

In [35]:
gics_df = hrp_utils.get_gics_data()
gics_df["date"] = pd.to_datetime(gics_df["date"])
gics_df["year"] = gics_df["date"].dt.year
gics_df["year_month"] = gics_df["date"].apply(lambda x: x.strftime("%Y-%m"))
gics_df.head()

df.shape: (18912, 6)


,date,ticker,sector,industry_group,industry,sub_industry,year,year_month
0,2000-12-31,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2000,2000-12
1,2001-12-31,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2001,2001-12
2,2002-12-31,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2002,2002-12
3,2003-12-31,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2003,2003-12
4,2004-12-31,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2004,2004-12


In [36]:
# getting last available gics data for a ticker
# gics_df = gics_df.drop_duplicates(["ticker"], keep="last").reset_index(drop=True)
gics_df = gics_df.drop(columns=["date", "year"])
gics_df.head()

,ticker,sector,industry_group,industry,sub_industry,year_month
0,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2000-12
1,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2001-12
2,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2002-12
3,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2003-12
4,AAL,Industrials,Transportation,Passenger Airlines,Passenger Airlines,2004-12


## Creating all dataframe with SNP constituents from 2000-2022

In [40]:
# Get returns for all tickers 
mega_df = pd.read_feather(returns_folder + "/all_file.feather")
mega_df = mega_df[mega_df.date > "1999-12-31"].reset_index(drop=True)
mega_df["mkt_cap"] = mega_df["shrout"] * mega_df["prc"]
mega_df = mega_df[["date", "ticker", "permno", "ret", "mkt_cap", "shrout", "prc"]]
mega_df["year"] = pd.to_datetime(mega_df["date"]).apply(lambda x:x.year)

In [41]:
with open(data_folder+"/snp_constituents.json") as f:
    snp_constitutents = json.load(f)

snp_constitutents = {str(int(x)-1): val for x, val in snp_constitutents.items()}
# snp_constitutents

In [42]:
df_list = list()

for yr in snp_constitutents:
    tickers = snp_constitutents[yr]
    sdf = mega_df[mega_df.year == int(yr)]
    sdf = sdf[sdf.ticker.isin(tickers)]
    df_list.append(sdf)

mega_df = pd.concat(df_list, axis=0, ignore_index=True)
mega_df["year_month"] = mega_df["date"].apply(lambda x: x.strftime("%Y-%m"))
mega_df

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,year_month
0,2000-01-03,ROK,84381.0,0.031332,9.392211e+06,190222.0,49.375000,2000,2000-01
1,2000-01-04,ROK,84381.0,-0.036709,9.047434e+06,190222.0,47.562500,2000,2000-01
2,2000-01-05,ROK,84381.0,0.005256,9.094989e+06,190222.0,47.812500,2000,2000-01
3,2000-01-06,ROK,84381.0,-0.016993,8.940434e+06,190222.0,47.000000,2000,2000-01
4,2000-01-07,ROK,84381.0,0.027926,9.190100e+06,190222.0,48.312500,2000,2000-01
...,...,...,...,...,...,...,...,...,...
2910009,2022-12-23,FAST,11618.0,0.001468,2.735502e+07,572760.0,47.759998,2022,2022-12
2910010,2022-12-27,FAST,11618.0,-0.005653,2.720037e+07,572760.0,47.490002,2022,2022-12
2910011,2022-12-28,FAST,11618.0,-0.013687,2.682808e+07,572760.0,46.840000,2022,2022-12
2910012,2022-12-29,FAST,11618.0,0.015372,2.724047e+07,572760.0,47.560001,2022,2022-12


In [43]:
# merging gics sectors
mega_df = mega_df.merge(gics_df, on=["ticker", "year_month"], how="left")
mega_df.head()

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,year_month,sector,industry_group,industry,sub_industry
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,2000-01,NaN,NaN,NaN,NaN
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,2000-01,NaN,NaN,NaN,NaN
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,2000-01,NaN,NaN,NaN,NaN
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,2000-01,NaN,NaN,NaN,NaN
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,2000-01,NaN,NaN,NaN,NaN


In [45]:
# assigning gics sectors monthwise - back fill if not available
gics_cols = ["sector", "industry_group", "industry", "sub_industry"]
mega_df[gics_cols] = mega_df.groupby(["permno"])[gics_cols].bfill().ffill().fillna("NA")
mega_df.head()

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,year_month,sector,industry_group,industry,sub_industry
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,2000-01,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,2000-01,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,2000-01,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,2000-01,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,2000-01,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment


In [46]:
mega_df.to_pickle(data_folder+"/all_data.pkl")

## Extracting Fin Ratios

In [47]:
fr_df_list = list()
for ticker in tickers:
    tfile = os.path.join(factors_folder, f"{ticker}.csv")
    tdf = pd.read_csv(tfile).iloc[:, 1:]
    fr_df_list.append(tdf)

fr_df = pd.concat(fr_df_list, axis=0, ignore_index=True).sort_values(["date", "ticker"]).reset_index(drop=True)
fr_df["date"] = pd.to_datetime(fr_df["date"])
# fr_df = fr_df.set_index(["date", "ticker"])
fr_df.head()

,ticker,date,pe_op_basic,pe_exi,pe_inc,bm
0,ADM,1986-08-31,14.050435,14.050435,14.612452,0.838902
1,CLX,1986-08-31,16.458333,16.928571,16.928571,0.425446
2,DAL,1986-08-31,36.228814,36.228814,36.228814,1.129183
3,IDXX,1986-08-31,-9.687500,-9.687500,-9.687500,0.092999
4,JKHY,1986-08-31,10.526308,10.526308,10.526308,0.633054


## Extracting Momentum

In [48]:
mm_df_list = list()
for ticker in tickers:
    tfile = os.path.join(mom_folder, f"{ticker}.feather")
    tdf = pd.read_feather(tfile)
    mm_df_list.append(tdf)
    
mm_df = pd.concat(mm_df_list, axis=0, ignore_index=True).sort_values(["date", "ticker"]).reset_index(drop=True)
mm_df["date"] = pd.to_datetime(mm_df["date"])
# mm_df = mm_df.set_index(["date", "ticker"])
mm_df.head()

,ticker,date,momentum,risk_adj_momentum
0,A,1993-01-29,-0.146358,-0.258332
1,AAL,1993-01-29,0.303296,0.535392
2,AAPL,1993-01-29,-0.029669,-0.052353
3,ABT,1993-01-29,-0.005231,-0.009217
4,ACN,1993-01-29,-0.258824,-0.456854


## Extracting Reversal

In [49]:
rev_df_list = list()
for ticker in tickers:
    tfile = os.path.join(reversal_folder, f"{ticker}.csv")
    tdf = pd.read_csv(tfile).iloc[:, 1:]
    rev_df_list.append(tdf)

rev_df = pd.concat(rev_df_list, axis=0, ignore_index=True).sort_values(["date", "ticker"]).reset_index(drop=True)
rev_df["date"] = pd.to_datetime(rev_df["date"])
rev_df.head()

,ticker,date,1M_reversal,1M_reversal*,risk_adj_1M_reversal,risk_adj_1M_reversal*
0,A,1993-01-29,0.003679,1.003692,-0.004936,1.753691
1,AAL,1993-01-29,-0.019231,0.981132,0.025722,1.714273
2,AAPL,1993-01-29,-0.043668,0.958159,0.058426,1.674134
3,ABT,1993-01-29,0.012195,1.012346,-0.016334,1.768810
4,ACN,1993-01-29,0.045455,1.047619,-0.060843,1.830441


## Creating factor data file

In [50]:
mm_df["year_month"] = mm_df["date"].apply(lambda x:x.strftime("%Y-%m"))
mm_df2 = mm_df.drop_duplicates(subset=["ticker", "year_month"], keep="last")[["ticker",  'momentum', 'risk_adj_momentum', 'year_month']]

rev_df["year_month"] = rev_df["date"].apply(lambda x:x.strftime("%Y-%m"))
rev_df2 = rev_df.drop_duplicates(subset=["ticker", "year_month"], keep="last")[["ticker", '1M_reversal', '1M_reversal*', 'risk_adj_1M_reversal',
       'risk_adj_1M_reversal*', 'year_month']]

In [51]:
fr_df["year_month"] = fr_df["date"].apply(lambda x:x.strftime("%Y-%m"))

In [52]:
factor_df = fr_df.merge(mm_df2, on=["year_month", "ticker"], how="left")
factor_df = factor_df.merge(rev_df2, on=["year_month", "ticker"], how="left").drop(columns=["year_month"])
factor_df

,ticker,date,pe_op_basic,pe_exi,pe_inc,bm,momentum,risk_adj_momentum,1M_reversal,1M_reversal*,risk_adj_1M_reversal,risk_adj_1M_reversal*
0,ADM,1986-08-31,14.050435,14.050435,14.612452,0.838902,NaN,NaN,NaN,NaN,NaN,NaN
1,CLX,1986-08-31,16.458333,16.928571,16.928571,0.425446,NaN,NaN,NaN,NaN,NaN,NaN
2,DAL,1986-08-31,36.228814,36.228814,36.228814,1.129183,NaN,NaN,NaN,NaN,NaN,NaN
3,IDXX,1986-08-31,-9.687500,-9.687500,-9.687500,0.092999,NaN,NaN,NaN,NaN,NaN,NaN
4,JKHY,1986-08-31,10.526308,10.526308,10.526308,0.633054,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
136119,WRK,2022-12-31,7.480851,9.739612,9.739612,1.802386,-0.148256,1.524394,-0.141576,0.875982,1.632574,0.951951
136120,XEL,2022-12-31,22.911765,22.911765,22.911765,0.605455,0.047905,-0.492708,-0.074721,0.930474,0.861581,1.011168
136121,XOM,2022-12-31,8.583658,9.004082,9.004082,0.576731,0.575840,-5.921412,-0.018971,0.981383,0.218659,1.066491
136122,ZBH,2022-12-31,36.849711,73.275862,96.590909,0.582915,0.002678,-0.027643,-0.145417,0.873044,1.676873,0.948758


## Normalising Factors

In [53]:
# factors were winsorized to a limit of 5
factor_cols = factor_df.columns[2:]
factor_df = factor_df[factor_df.date >= "2000-01-01"].reset_index(drop=True)
factor_df[factor_cols] = factor_df.groupby("date")[factor_cols].apply(lambda x: regression.normalize_df(x, factor_cols))
factor_df

,ticker,date,pe_op_basic,pe_exi,pe_inc,bm,momentum,risk_adj_momentum,1M_reversal,1M_reversal*,risk_adj_1M_reversal,risk_adj_1M_reversal*
0,AAPL,2000-01-31,0.903453,1.059684,1.111897,-0.133413,3.352743,3.352743,0.506104,0.560556,-0.506104,0.560926
1,ABT,2000-01-31,0.672642,0.577095,0.605530,-0.508574,-0.374969,-0.374969,0.220229,0.233473,-0.220229,0.233843
2,ACN,2000-01-31,0.462862,0.404520,0.424452,0.518873,-0.265712,-0.265712,-0.544054,-0.517501,0.544054,-0.517131
3,ADBE,2000-01-31,1.049993,1.133691,1.189550,-0.613762,3.959185,3.959185,0.182601,0.192497,-0.182601,0.192867
4,ADI,2000-01-31,4.701288,4.571352,4.796593,-0.393590,3.878180,3.878180,-2.201146,-1.712210,2.201146,-1.711840
...,...,...,...,...,...,...,...,...,...,...,...,...
100980,WRK,2022-12-31,-0.699783,-0.576338,-0.539753,3.508156,-0.352376,0.352376,-0.570452,-0.546475,0.570452,-0.546475
100981,XEL,2022-12-31,0.246076,0.136557,0.174105,0.712165,0.285586,-0.285586,0.147923,0.150521,-0.147923,0.150521
100982,XOM,2022-12-31,-0.632185,-0.616146,-0.579615,0.645066,2.002560,-2.002560,0.746968,0.801673,-0.746968,0.801673
100983,ZBH,2022-12-31,1.100421,2.862331,4.167112,0.659512,0.138497,-0.138497,-0.611728,-0.584051,0.611728,-0.584051


In [57]:
# factor_df.to_feather(processed_data_folder + "/factor_data.feather")

## Creating Cluster Data

In [54]:
ret_df = pd.read_pickle(ret_file)
ret_df = ret_df.groupby("year").apply(lambda x: hrp_utils.clean_dataset(x)).reset_index(drop=True)
ret_df.head()

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,sector,industry_group,industry,sub_industry
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment


## Creating clusters from years 2001 - 2022 on a rolling basis

In [58]:
cluster_sizes = [5, 10, 20, 35, 50, 100]

In [59]:
%%time
cluster_df = ret_df.groupby("year").apply(lambda x: hrp_utils.perform_clustering(x, cluster_sizes)).reset_index(drop=True)
cluster_df.head()

CPU times: user 7.69 s, sys: 949 ms, total: 8.64 s
Wall time: 10.5 s


,date,ticker,permno,ret,mkt_cap,shrout,prc,year,sector,industry_group,industry,sub_industry,cluster_5,cluster_10,cluster_20,cluster_35,cluster_50,cluster_100
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,5,4,4,14,43
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,5,4,4,14,43
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,5,4,4,14,43
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,5,4,4,14,43
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,5,4,4,14,43


## Assigining clusters to forward years

In [60]:
cluster_df_lst = list()
cdf = cluster_df.copy()
cdf.shape

(2854829, 18)

In [61]:
%%time
for n in cluster_sizes:
    out = list()
    for year in range(2000, 2022):
        ydf = cluster_df[cluster_df["year"] == year]
        ticker_cluster_map = ydf[["ticker", f"cluster_{n}"]].drop_duplicates().set_index("ticker")[f"cluster_{n}"].to_dict()
        
        ydf = cluster_df[cluster_df["year"] == year + 1]
        ydf[f"fwd_cluster_{n}"] = ydf["ticker"].map(ticker_cluster_map)
        ydf = ydf[["date", "ticker", f"fwd_cluster_{n}"]]
        out.append(ydf)
    
    ydf = pd.concat(out, axis=0, ignore_index=True)            
    cdf = cdf.merge(ydf, on=["date", "ticker"], how="left")

CPU times: user 8.43 s, sys: 2.71 s, total: 11.1 s
Wall time: 12.9 s


In [63]:
# cdf.to_pickle(processed_data_folder + "/agglo_clusters.pkl")

In [64]:
cdf.head()

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,sector,industry_group,...,cluster_20,cluster_35,cluster_50,cluster_100,fwd_cluster_5,fwd_cluster_10,fwd_cluster_20,fwd_cluster_35,fwd_cluster_50,fwd_cluster_100
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,Industrials,Capital Goods,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,Industrials,Capital Goods,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,Industrials,Capital Goods,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,Industrials,Capital Goods,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,Industrials,Capital Goods,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN


## Creating Monthly Return Timeseries

In [87]:
# reading monthly factor data
factor_df = pd.read_feather(factor_file)
factor_df["year_month"] = factor_df["date"].apply(lambda x:x.strftime("%Y-%m"))
factor_df.head()

,ticker,date,pe_op_basic,pe_exi,pe_inc,bm,momentum,risk_adj_momentum,1M_reversal,1M_reversal*,risk_adj_1M_reversal,risk_adj_1M_reversal*,year_month
0,AAPL,2000-01-31,0.903453,1.059684,1.111897,-0.133413,3.352743,3.352743,0.506104,0.560556,-0.506104,0.560926,2000-01
1,ABT,2000-01-31,0.672642,0.577095,0.605530,-0.508574,-0.374969,-0.374969,0.220229,0.233473,-0.220229,0.233843,2000-01
2,ACN,2000-01-31,0.462862,0.404520,0.424452,0.518873,-0.265712,-0.265712,-0.544054,-0.517501,0.544054,-0.517131,2000-01
3,ADBE,2000-01-31,1.049993,1.133691,1.189550,-0.613762,3.959185,3.959185,0.182601,0.192497,-0.182601,0.192867,2000-01
4,ADI,2000-01-31,4.701288,4.571352,4.796593,-0.393590,3.878180,3.878180,-2.201146,-1.712210,2.201146,-1.711840,2000-01


In [88]:
# reading cluster daily data
cluster_df = pd.read_pickle(cluster_file)
cluster_df["year_month"] = cluster_df["date"].apply(lambda x:x.strftime("%Y-%m"))
cluster_df.head()

,date,ticker,permno,ret,mkt_cap,shrout,prc,year,sector,industry_group,...,cluster_35,cluster_50,cluster_100,fwd_cluster_5,fwd_cluster_10,fwd_cluster_20,fwd_cluster_35,fwd_cluster_50,fwd_cluster_100,year_month
0,2000-01-03,ROK,84381.0,0.031332,9392211.250,190222.0,49.3750,2000,Industrials,Capital Goods,...,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN,2000-01
1,2000-01-04,ROK,84381.0,-0.036709,9047433.875,190222.0,47.5625,2000,Industrials,Capital Goods,...,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN,2000-01
2,2000-01-05,ROK,84381.0,0.005256,9094989.375,190222.0,47.8125,2000,Industrials,Capital Goods,...,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN,2000-01
3,2000-01-06,ROK,84381.0,-0.016993,8940434.000,190222.0,47.0000,2000,Industrials,Capital Goods,...,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN,2000-01
4,2000-01-07,ROK,84381.0,0.027926,9190100.375,190222.0,48.3125,2000,Industrials,Capital Goods,...,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN,2000-01


In [89]:
daily_df = cluster_df[["date", "ticker", "ret"]]

In [90]:
%%time
monthly_df = hrp_utils.get_monthly_returns(daily_df)
monthly_df.head()

CPU times: user 15.6 s, sys: 514 ms, total: 16.1 s
Wall time: 16.1 s


,ticker,date,ret
0,A,2000-01-31,-0.143897
1,A,2000-02-29,0.569405
2,A,2000-03-31,0.001203
3,A,2000-04-30,-0.147837
4,A,2000-05-31,-0.169252


In [91]:
monthly_df["year_month"] = monthly_df["date"].apply(lambda x:x.strftime("%Y-%m"))
monthly_df.head()

,ticker,date,ret,year_month
0,A,2000-01-31,-0.143897,2000-01
1,A,2000-02-29,0.569405,2000-02
2,A,2000-03-31,0.001203,2000-03
3,A,2000-04-30,-0.147837,2000-04
4,A,2000-05-31,-0.169252,2000-05


In [92]:
# getting required data from cluster df
req_cluster_cols = ['ticker', 'permno', 'year_month', 'year', 'mkt_cap', 'sector', 'industry_group', 'industry', 'sub_industry', 'cluster_5', 'cluster_10', 'cluster_20',
                    'cluster_35', 'cluster_50', 'cluster_100','fwd_cluster_5', 'fwd_cluster_10', 'fwd_cluster_20', 'fwd_cluster_35', 
                    'fwd_cluster_50', 'fwd_cluster_100']

simplified_cluster_df = (cluster_df.drop_duplicates(subset=["ticker", "year_month"], keep="last"))[req_cluster_cols].reset_index(drop=True)
simplified_cluster_df.head()

,ticker,permno,year_month,year,mkt_cap,sector,industry_group,industry,sub_industry,cluster_5,...,cluster_20,cluster_35,cluster_50,cluster_100,fwd_cluster_5,fwd_cluster_10,fwd_cluster_20,fwd_cluster_35,fwd_cluster_50,fwd_cluster_100
0,ROK,84381.0,2000-01,2000,9.390604e+06,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
1,ROK,84381.0,2000-02,2000,8.587047e+06,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
2,ROK,84381.0,2000-03,2000,7.934716e+06,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
3,ROK,84381.0,2000-04,2000,7.472154e+06,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN
4,ROK,84381.0,2000-05,2000,7.708943e+06,Industrials,Capital Goods,Electrical Equipment,Electrical Components & Equipment,0,...,4,4,14,43,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# adding clusters
monthly_df = monthly_df.merge(simplified_cluster_df, on=["ticker", "year_month"], how="left")
monthly_df.head()

,ticker,date,ret,year_month,permno,year,mkt_cap,sector,industry_group,industry,...,cluster_20,cluster_35,cluster_50,cluster_100,fwd_cluster_5,fwd_cluster_10,fwd_cluster_20,fwd_cluster_35,fwd_cluster_50,fwd_cluster_100
0,A,2000-01-31,-0.143897,2000-01,87432.0,2000,29916750.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,5,34,16,96,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2000-02-29,0.569405,2000-02,87432.0,2000,46951500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,5,34,16,96,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2000-03-31,0.001203,2000-03,87432.0,2000,47008000.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,5,34,16,96,NaN,NaN,NaN,NaN,NaN,NaN
3,A,2000-04-30,-0.147837,2000-04,87432.0,2000,40058500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,5,34,16,96,NaN,NaN,NaN,NaN,NaN,NaN
4,A,2000-05-31,-0.169252,2000-05,87432.0,2000,33278500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,5,34,16,96,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# getting required factors
req_factors = ['pe_exi', 'bm', 'momentum', '1M_reversal']
simplified_factor_df = factor_df[["ticker", "year_month"] + req_factors]
simplified_factor_df.head()

,ticker,year_month,pe_exi,bm,momentum,1M_reversal
0,AAPL,2000-01,1.059684,-0.133413,3.352743,0.506104
1,ABT,2000-01,0.577095,-0.508574,-0.374969,0.220229
2,ACN,2000-01,0.404520,0.518873,-0.265712,-0.544054
3,ADBE,2000-01,1.133691,-0.613762,3.959185,0.182601
4,ADI,2000-01,4.571352,-0.393590,3.878180,-2.201146


In [95]:
# adding factors
monthly_df = monthly_df.merge(simplified_factor_df, on=["ticker", "year_month"], how="left")
monthly_df.head()

,ticker,date,ret,year_month,permno,year,mkt_cap,sector,industry_group,industry,...,fwd_cluster_5,fwd_cluster_10,fwd_cluster_20,fwd_cluster_35,fwd_cluster_50,fwd_cluster_100,pe_exi,bm,momentum,1M_reversal
0,A,2000-01-31,-0.143897,2000-01,87432.0,2000,29916750.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2000-02-29,0.569405,2000-02,87432.0,2000,46951500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2000-03-31,0.001203,2000-03,87432.0,2000,47008000.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,NaN,NaN,NaN,NaN,NaN,NaN,3.100619,-0.438389,5.000000,-2.523678
3,A,2000-04-30,-0.147837,2000-04,87432.0,2000,40058500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,NaN,NaN,NaN,NaN,NaN,NaN,2.489225,-0.463607,5.000000,0.475024
4,A,2000-05-31,-0.169252,2000-05,87432.0,2000,33278500.0,Healthcare,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,...,NaN,NaN,NaN,NaN,NaN,NaN,2.174267,-0.419256,3.906034,-0.169328


In [96]:
# monthly_df.to_pickle(processed_data_folder + "/monthly_data.pkl")

## Daily Performance Analysis from 2018-2022

In [ ]:
daily